# Figure 4 - Extraction Optimization

In [1]:
import pandas as pd
from bokeh.io import show, export_svg
from bokeh.plotting import output_notebook
from bokeh.models import Line, Div
from bokeh.layouts import row, column

In [2]:
output_notebook()

Loading BokehJS ...

## Load Data

#### MzML Loader

In [3]:
from massdash.loaders import MzMLDataLoader
loader45 = MzMLDataLoader(rsltsFile="figure4_report_diann.tsv",
                        dataFiles="200113_AUR_diaPASEF_HY_200ng_45ng_90min_Slot1-6_1_1637.mzML",
                        libraryFile="191221_2proteome_lib_decoy.pqp",
                        rsltsFileType="DIA-NN")
 

[2024-01-17 13:41:51,731] MzMLDataAccess - INFO - Opening 200113_AUR_diaPASEF_HY_200ng_45ng_90min_Slot1-6_1_1637.mzML file...: Elapsed 57.898640394210815 ms
[2024-01-17 13:41:51,735] MzMLDataAccess - INFO - There are 257835 spectra and 0 chromatograms.
[2024-01-17 13:41:52,708] MzMLDataAccess - INFO - There are 3967 MS1 spectra and 253868 MS2 spectra.


#### Target Peptides

In [4]:
pep1 = "NPSFSEEPWLEIK"
pep2 = "NAFLESELDEKENLLESVQR"

---

#### Pannel 1: Initial Extraction Window with a im_extraction window of 0.06

In [5]:
from massdash.plotting import InteractivePlotter, PlotConfig
from massdash.peakPickers import MRMTransitionGroupPicker
from massdash.structs import TargetedDIAConfig

extraction = TargetedDIAConfig()
extraction.mz_tol = 20
extraction.im_window = 0.06
extraction.rt_window = 50

tg = loader45.loadTransitionGroups(pep2, 3, extraction)['200113_AUR_diaPASEF_HY_200ng_45ng_90min_Slot1-6_1_1637.mzML']

## peak picking
picker = MRMTransitionGroupPicker(smoother='sgolay', sgolay_frame_length=7, sgolay_polynomial_order=3)
picker.setGeneralParameters(use_precursors='true')
feats = picker.pick(tg)

plotConfig = PlotConfig()
plotConfig.smoothing_dict = dict(type='none')
plotConfig.title = 'Ion Mobility Extraction 0.06'
plotter = InteractivePlotter(plotConfig)


fig = plotter.plot(tg, [feats[1]])

fig.output_backend = 'svg'
show(plotter.fig)

fig1 = fig

torch is not installed. Please install it using 'pip install torch'.
torchmetrics is not installed. Please install it using 'pip install torchmetrics'.


2024-01-17 13:41:59.176 
  command:

    streamlit run /home/roestlab/anaconda3/envs/massDash_josh/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


#### Panel 2: 2D Heatmap

##### MS1 2D Heatmap

In [6]:
from massdash.structs import TargetedDIAConfig

extraction = TargetedDIAConfig()
extraction.im_window = 0.1
extraction.rt_window = 50
extraction.mz_tol = 20
features = loader45.loadFeatureMaps(pep2, 3, extraction)['200113_AUR_diaPASEF_HY_200ng_45ng_90min_Slot1-6_1_1637.mzML']

In [7]:
from massdash.plotting import PlotConfig, InteractiveTwoDimensionPlotter

config = PlotConfig()

config.aggregate_mslevels = True
config.include_ms1 = True
config.include_ms2 = False
config.type_of_heatmap = 'retention time vs ion mobility'
config.normalization_dict = {'type':'equalization', 'bins':2}
config.smoothing_dict = {'type':'gauss', 'sigma':1.2}
plotter = InteractiveTwoDimensionPlotter(config)

fig = plotter.plot(features)[0]
fig.output_backend = 'svg'
rt = 3958.035278
fig.line([rt - extraction.rt_window /2, rt + extraction.rt_window /2], [1.049, 1.049], line_width=2, color='black')
fig.line([rt - extraction.rt_window /2, rt + extraction.rt_window /2], [1.066, 1.066], line_width=2, color='black')
plotter.show()

fig2a = plotter.fig[0]

Loading BokehJS ...

##### MS2 2D Heatmap

In [8]:
from massdash.plotting import PlotConfig, InteractiveTwoDimensionPlotter

config = PlotConfig()

config.aggregate_mslevels = True
config.include_ms1 = False
config.include_ms2 = True
config.type_of_heatmap = 'retention time vs ion mobility'

config.normalization_dict = {'type':'equalization', 'bins':2}
config.smoothing_dict = {'type':'gauss', 'sigma':1.2}
plotter = InteractiveTwoDimensionPlotter(config)

fig = plotter.plot(features)[0]
fig.output_backend = 'svg'
rt = 3958.035278
fig.line([rt - extraction.rt_window /2, rt + extraction.rt_window /2], [1.049, 1.049], line_width=2, color='black')
fig.line([rt - extraction.rt_window /2, rt + extraction.rt_window /2], [1.066, 1.066], line_width=2, color='black')
plotter.show()

fig2b = plotter.fig[0]

Loading BokehJS ...

#### Panel 3: Updated 1D chromatogram

In [9]:
from massdash.plotting import InteractivePlotter, PlotConfig

extraction = TargetedDIAConfig()
extraction.mz_tol = 20
extraction.im_start = 1.049
extraction.im_end = 1.066
extraction.rt_window = 50

tg = loader45.loadTransitionGroups(pep2, 3, extraction)['200113_AUR_diaPASEF_HY_200ng_45ng_90min_Slot1-6_1_1637.mzML']


## peak picking
picker = MRMTransitionGroupPicker(smoother='sgolay', sgolay_frame_length=7, sgolay_polynomial_order=3)
picker.setGeneralParameters(use_precursors='true')
feats = picker.pick(tg)

plotConfig = PlotConfig()
plotConfig.smoothing_dict = dict(type='none')
plotConfig.title = "Ion Mobility Extraction 1.049 - 1.066"
plotter = InteractivePlotter(plotConfig)


fig = plotter.plot(tg, [feats[0]])
fig.output_backend = 'svg'

show(plotter.fig)

fig3 = fig

---

#### Plots Together 

In [10]:
#### Remove Toolbars
fig1.toolbar_location = None
fig2a.toolbar_location = None
fig2b.toolbar_location = None
fig3.toolbar_location = None

fig1.output_backend = 'svg'
fig2a.output_backend = 'svg'
fig2b.output_backend = 'svg'
fig3.output_backend = 'svg'

# make titles not bold
fig1.title.text_font_style = 'normal'
fig2a.title.text_font_style = 'normal'
fig2b.title.text_font_style = 'normal'
fig3.title.text_font_style = 'normal'

# Make fonts bigger
fig1.axis.axis_label_text_font_size= '14pt'
fig1.axis.major_label_text_font_size = '14pt'

fig2a.axis.axis_label_text_font_size= '14pt'
fig2a.axis.major_label_text_font_size = '14pt'

fig2b.axis.axis_label_text_font_size= '14pt'
fig2b.axis.major_label_text_font_size = '14pt'

fig3.axis.axis_label_text_font_size= '14pt'
fig3.axis.major_label_text_font_size = '14pt'

fig1.legend.label_text_font_size = '12pt'
fig1.legend.title_text_font_size = '12pt'

fig3.legend.label_text_font_size = '12pt'
fig3.legend.title_text_font_size = '12pt'

## adjust limits
fig1.y_range.end  = 3500
fig1.width = 800 
fig1.height= 350 
 
fig2a.width = 375
fig2a.height= 400 
  
fig2b.width = 375
fig2b.height= 400 
 
fig3.width = 800 
fig3.height= 350 
 
letter_style = {
    'font-size': '24px',
    'margin-top': '-10px',  # Adjust this value
}

letter1 = Div(text='<span style="font-weight:normal;">A</span>', width=20, height=50, style=letter_style)
letter2a = Div(text='<span style="font-weight:normal;">B</span>', width=20, height=50, style=letter_style)
letter2b = Div(text='<span style="font-weight:normal;">C</span>', width=20, height=50, style=letter_style)
letter3 = Div(text='<span style="font-weight:normal;">D</span>', width=20, height=50, style=letter_style)

fig_tog = column(row(letter1, fig1), row(letter2a, fig2a, letter2b, fig2b), row(letter3, fig3))

show(fig_tog)


In [11]:
export_png(fig_tog, filename='Figure_Extraction_Optimization.png')

NameError: name 'export_png' is not defined